In [1]:
import sys
!{sys.executable} -m pip install rdflib yatter morph-kgc

  Obtaining dependency information for morph-kgc from https://files.pythonhosted.org/packages/ea/4a/58e493fcfe35cfac4b2312f2d246e3f404956086cf7125b1ffc8ef1508ef/morph_kgc-2.6.3-py3-none-any.whl.metadata
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.0/46.0 kB 3.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 15.2/15.2 MB 6.7 MB/s eta 0:00:0000:0100:01
  Obtaining dependency information for elementpath<5.0.0,>=4.0.0 from https://files.pythonhosted.org/packages/eb/4f/fecd0fc5adc5f91a6154189ea7b2192cf62ce7d685a0e9b23737acb31808/elementpath-4.1.5-py3-none-any.whl.metadata
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.1/9.1 MB 7.3 MB/s eta 0:00:0000:0100:01
  Obtaining dependency information for pyoxigraph<0.4.0,>=0.3.0 from https://files.pythonhosted.org/packages/30/a4/1e820e3f4764876457eda24597ae1585cf4a9ab1c87aa00bec9f384a4f0f/pyoxigraph-0.3.19-cp311-cp311-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata
  Obtaining dependency information for sql-metadata<3.0.

In [2]:
from os import getcwd
from os.path import join, dirname
folder = dirname(getcwd())

In [3]:
from rdflib import Graph

# Load RDF data from file
g = Graph()
rdf_file = join(folder, "rdf/digital-competence-framework-skos-ap-eu.rdf")
g.parse(rdf_file, format="xml")

# SPARQL query
sparql_query = """
    PREFIX rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#>
    PREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#>
    PREFIX skos: <http://www.w3.org/2004/02/skos/core#>
    PREFIX dc: <http://purl.org/dc/elements/1.1/>
    
    SELECT DISTINCT ?label ?id
    WHERE {
      ?s skos:prefLabel ?label ;
          dc:identifier ?id .
      FILTER (!CONTAINS(?label, 'Level'))
    }
    ORDER BY ?label
"""

# Execute the SPARQL query
results = g.query(sparql_query)

# Print the query results
for row in results:
    label = str(row.label)[:3].strip()
    if len(label) == 2:
        label = label[:1]
    print(f"{label},{row.id}")

1,8b23nt41l4
1.1,9i6ta81748
1.2,3h9eab1489
1.3,4s23kdq383
2,2k3209g1wc
2.1,8ys96p08p4
2.2,9a5d39t62b
2.3,1dz4n8r554
2.4,y2s2861s9p
2.5,99vvo91542
2.6,7d4t7002v7
3,4b465z8p2e
3.1,u69o196gu6
3.2,sq0c9q6116
3.3,2mi55kl065
3.4,0690r7o8r2
4,1fz48677f4
4.1,aa985fr035
4.2,1b043jw49b
4.3,mg54my1580
4.4,o9e8s72z21
5,86xdxt8620
5.1,13z458t54u
5.2,00b51s9rc5
5.3,3bgx2a6278
5.4,860966ekgo


In [4]:
import yatter
from ruamel.yaml import YAML

# YARRRML to RML
yaml = YAML(typ='safe', pure=True)
mappings_file = join(folder, "mappings/corso-kg-yarrrml.yml")
rml_content = yatter.translate(yaml.load(open(mappings_file)))

# Print generated RML triples
print(rml_content)

# Save RML to file
rml_file = join(folder, "mappings/corso-kg-rml.ttl")
with open(rml_file, "w") as rml_output:
    rml_output.write(rml_content)

2023-10-20 16:30:12,922 | INFO: Translating YARRRML mapping to [R2]RML
2023-10-20 16:30:12,923 | INFO: RML content is created!
2023-10-20 16:30:12,934 | INFO: Mapping has been syntactically validated.
2023-10-20 16:30:12,935 | INFO: Translation has finished successfully.


@prefix kg: <http://www.cefriel.com/corso-kg/data/>.
@prefix ex: <http://www.cefriel.com/corso-kg/schema/>.
@prefix skos: <http://www.w3.org/2004/02/skos/core#>.
@prefix rr: <http://www.w3.org/ns/r2rml#>.
@prefix rml: <http://semweb.mmlab.be/ns/rml#>.
@prefix rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#>.
@prefix rdfs: <http://www.w3.org/2000/01/rdf-schema#>.
@prefix xsd: <http://www.w3.org/2001/XMLSchema#>.
@prefix ql: <http://semweb.mmlab.be/ns/ql#>.
@prefix d2rq: <http://www.wiwiss.fu-berlin.de/suhl/bizer/D2RQ/0.1#>.
@prefix foaf: <http://xmlns.com/foaf/0.1/>.
@prefix schema: <http://schema.org/>.
@prefix formats: <http://www.w3.org/ns/formats/>.
@prefix comp: <http://semweb.mmlab.be/ns/rml-compression#>.
@prefix void: <http://rdfs.org/ns/void#>.
@prefix fnml: <http://semweb.mmlab.be/ns/fnml#>.
@prefix grel: <http://users.ugent.be/~bjdmeest/function/grel.ttl#>.
@base <http://example.com/ns#>.


<unit_0> a rr:TriplesMap;

	rml:logicalSource [
		a rml:LogicalSource;
		rml:source 

In [5]:
import morph_kgc

config = """      
            [Competences]
            mappings: ./../mappings/corso-kg-rml.ttl
         """
# Generate the triples and load them to an RDFLib graph
g_rdflib = morph_kgc.materialize(config)

mapping_file = join(folder, "mappings/corso-kg-rml.ttl")
g_mappings = Graph()
g_mappings.parse(mapping_file, format='ttl')

for ns_prefix, namespace in g_mappings.namespaces():
    g_rdflib.bind(ns_prefix, namespace)

# Save to file
output_file = join(folder, "rdf/knowledge-graph.ttl")
g_rdflib.serialize(destination=output_file, format='ttl', encoding="utf-8")

2023-10-20 16:30:26,047 | INFO: NumExpr defaulting to 8 threads.
2023-10-20 16:30:26,770 | DEBUG: CONFIGURATION: {'output_file': 'knowledge-graph', 'na_values': ',nan', 'safe_percent_encoding': '', 'read_parsed_mappings_path': '', 'write_parsed_mappings_path': '', 'mapping_partitioning': 'PARTIAL-AGGREGATIONS', 'logging_file': '', 'oracle_client_lib_dir': '', 'oracle_client_config_dir': '', 'udfs': '', 'output_dir': '', 'output_format': 'N-TRIPLES', 'only_printable_chars': 'no', 'infer_sql_datatypes': 'no', 'logging_level': 'INFO', 'number_of_processes': '16'}
2023-10-20 16:30:26,771 | DEBUG: DATA SOURCE `Competences`: {'mappings': './../mappings/corso-kg-rml.ttl'}
2023-10-20 16:30:27,581 | DEBUG: Removed self-join from mapping rule `#TM6`.
2023-10-20 16:30:27,587 | INFO: 12 mapping rules retrieved.
2023-10-20 16:30:27,601 | DEBUG: All predicate maps are constant-valued, invariant subset is not enforced.
2023-10-20 16:30:27,609 | DEBUG: All graph maps are constant-valued, invariant sub

<Graph identifier=N8bc4573e67124a4c978d08bff3343643 (<class 'rdflib.graph.Graph'>)>

In [6]:
q_res = g_rdflib.query('SELECT (COUNT(*) AS ?num_triples) WHERE { ?s ?p ?o }')

# Print the query results
for row in q_res:
    print(f"Number of triples generated: {row.num_triples}")

Number of triples generated: 1488


In [9]:
ontology_file = join(folder, "rdf/digital-competence-framework-skos-ap-eu.rdf")
g_rdflib.parse(ontology_file, format='xml')

<Graph identifier=N8bc4573e67124a4c978d08bff3343643 (<class 'rdflib.graph.Graph'>)>

In [10]:
q_res = g_rdflib.query('SELECT (COUNT(*) AS ?num_triples) WHERE { ?s ?p ?o }')

# Print the query results
for row in q_res:
    print(f"Number of triples generated: {row.num_triples}")

Number of triples generated: 9472


In [43]:
sparql_query = """
PREFIX cefriel: <http://www.cefriel.com/corso-kg/schema/>
PREFIX foaf: <http://xmlns.com/foaf/0.1/>
PREFIX skos: <http://www.w3.org/2004/02/skos/core#>

SELECT ?label ?level
WHERE {
    ?employee foaf:name "Jane" .
    ?employee foaf:surname "Brown" .
    ?ec a cefriel:EmployeeCompetence ;
        cefriel:employee ?employee ;
        cefriel:competence ?c ;
        cefriel:level ?level .
    ?c skos:prefLabel ?label . 
}
"""

In [44]:
results = g_rdflib.query(sparql_query)
# Print the query results
for row in results:
    print(f"Competence [{row.label}]\tLevel [{row.level}]")

Competence [2.1 Interacting through digital technologies]	Level [4]
Competence [4.2 Protecting personal data and privacy]	Level [5]
Competence [4.4 Protecting the environment]	Level [3]
Competence [1.3 Managing data, information and digital content]	Level [3]
Competence [5.1 Solving technical problems]	Level [3]
Competence [5.3 Creatively using digital technologies]	Level [1]
Competence [4.3 Protecting health and well-being]	Level [4]
Competence [3.4 Programming]	Level [2]
Competence [3.3 Copyright and licences]	Level [5]
Competence [2.5 Netiquette]	Level [3]
